In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.models import load_model
from skimage.transform import resize, rotate
import numpy as np
import cv2
import seaborn as sns
import pandas as pd
import matplotlib as plt

In [2]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2,
    )
# test_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
x_train = train_datagen.flow_from_directory(
    'Dataset4/training_set',
    target_size=(64,64),
    batch_size=2862,
    save_to_dir='Dataset4/aug_set',
    class_mode='categorical',
    color_mode="grayscale",
    subset="training",
    )
x_test = train_datagen.flow_from_directory(
    'Dataset4/training_set',
    target_size=(64,64),
    batch_size=1,
    class_mode='categorical',
    color_mode="grayscale",
    subset="validation",
    )
# x_test = train_datagen.flow_from_directory('Dataset/test_set', target_size=(64,64), batch_size=200, class_mode='categorical', color_mode="grayscale")

Found 6872 images belonging to 9 classes.
Found 1714 images belonging to 9 classes.


In [4]:
model = Sequential()

In [5]:
model.add(Convolution2D(32,(3,3),input_shape=(64,64,1),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(units=512,activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(units=9,activation='softmax'))

In [6]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
# model.fit(x_train, steps_per_epoch=24, epochs=10, validation_data=x_test, validation_steps=40)
track = model.fit(
    x_train,
    steps_per_epoch=3,
    epochs=5,
    validation_data=x_test,
    validation_steps=1,
    )

Epoch 1/5
3/3 [==============================] - 174s 48s/step - loss: 2.5709 - accuracy: 0.1345 - val_loss: 1.7873 - val_accuracy: 0.0000e+00
Epoch 2/5
3/3 [==============================] - 160s 38s/step - loss: 2.3539 - accuracy: 0.1409 - val_loss: 1.9821 - val_accuracy: 1.0000
Epoch 3/5
3/3 [==============================] - 166s 40s/step - loss: 2.1897 - accuracy: 0.1307 - val_loss: 2.3236 - val_accuracy: 0.0000e+00
Epoch 4/5
3/3 [==============================] - 164s 38s/step - loss: 2.1707 - accuracy: 0.1492 - val_loss: 1.9058 - val_accuracy: 0.0000e+00
Epoch 5/5
3/3 [==============================] - 168s 42s/step - loss: 2.1520 - accuracy: 0.1895 - val_loss: 2.1733 - val_accuracy: 0.0000e+00


In [8]:
model.save('weight5.h5')

In [13]:
model = load_model('weight3.h5')

In [14]:
model.fit(x_train, steps_per_epoch=45, epochs=10, validation_data=x_test, validation_steps=9)

Epoch 1/10
45/45 [==============================] - 84s 2s/step - loss: 0.0068 - accuracy: 0.9990 - val_loss: 0.3288 - val_accuracy: 0.9728
Epoch 2/10
45/45 [==============================] - 81s 2s/step - loss: 0.0051 - accuracy: 0.9993 - val_loss: 0.2563 - val_accuracy: 0.9717
Epoch 3/10
45/45 [==============================] - 80s 2s/step - loss: 0.0049 - accuracy: 0.9991 - val_loss: 0.2908 - val_accuracy: 0.9722
Epoch 4/10
45/45 [==============================] - 75s 2s/step - loss: 0.0034 - accuracy: 0.9995 - val_loss: 0.3248 - val_accuracy: 0.9733
Epoch 5/10
45/45 [==============================] - 90s 2s/step - loss: 0.0038 - accuracy: 0.9992 - val_loss: 0.2781 - val_accuracy: 0.9706
Epoch 6/10
45/45 [==============================] - 102s 2s/step - loss: 0.0024 - accuracy: 0.9997 - val_loss: 0.2732 - val_accuracy: 0.9667
Epoch 7/10
45/45 [==============================] - 118s 3s/step - loss: 0.0033 - accuracy: 0.9990 - val_loss: 0.3111 - val_accuracy: 0.9739
Epoch 8/10
45/45 [

In [4]:
def detect(frame):
    img = resize(frame,(64,64,1))
    # cv2.imshow("frame2",img)
    img = np.expand_dims(img, axis=0)
    if np.max(img) > 1:
        img = img/255
    print(img.shape)
    prediction = model.predict(img)
    print(prediction)
    prediction = np.where(prediction[:] > 0.5)[1] + 1
    print(prediction[0])
    # # frame=rotate(frame,45)
    # # cv2.imshow("frame",frame)
    # # print(frame.shape)
    # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # thresh, frame = cv2.threshold(frame, 126, 255, cv2.THRESH_BINARY)
    # print(frame.shape)
    # kernel = np.ones((5,5),np.uint8)
    # frame = cv2.morphologyEx(frame, cv2.MORPH_CLOSE, kernel)
    # frame = cv2.morphologyEx(frame, cv2.MORPH_OPEN, kernel)
    # # contours, _ = cv2.findContours(frame, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    # # contours = sorted(contours, key=cv2.contourArea, reverse=True)
    # # print(contours[1:])
    # # cv2.drawContours(frame, contours, -1, (0,255,0), 3)
    # cv2.imshow("frame1",frame)
    # cv2.imshow("frame2",img)
    # cv2.waitKey(0) 

In [6]:
frame = cv2.imread(r"Dataset\\test_set\\C\\150.png")
# frame = cv2.imread(r"Dataset\\test_set\B\\197.png")
data = detect(frame)

(1, 64, 64, 1)
1/1 [==============================] - 1s 554ms/step
[[1.4358401e-10 1.2662126e-06 9.9398309e-01 2.2878355e-07 2.4062178e-06
  3.8932950e-09 6.0052965e-03 7.7619898e-06 1.3038267e-09]]
3


In [11]:
# cv2.destroyAllWindows()

In [15]:
x_train[0][0].shape

(350, 64, 64, 1)